In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('pred').getOrCreate()

In [4]:
data = spark.read.csv(r"C:\Users\Asus\Desktop\onlinefoods.csv", inferSchema = True, header = True)

In [5]:
from pyspark.sql.functions import mean


In [8]:
data

DataFrame[Age: int, Gender: string, Marital Status: string, Occupation: string, Monthly Income: string, Educational Qualifications: string, Family size: int, latitude: double, longitude: double, Pin code: int, Output: string, Feedback: string, _c12: string]

NameError: name 'new_data' is not defined

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

In [ ]:
new_data['_c12'] = new_data['_c12'].astype('int')

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.ml import Pipeline

In [ ]:
indexers = []
encoders = []

for column in data.columns:
    # Sprawdzenie typu kolumny
    if not isinstance(data.schema[column].dataType, (IntegerType, DoubleType)):
        # Utworzenie StringIndexera dla kolumny
        indexer = StringIndexer(inputCol=column, outputCol=f"{column}_indx")
        indexers.append(indexer)
        
        # Utworzenie OneHotEncodera dla wyniku StringIndexera
        encoder = OneHotEncoder(inputCols=[f"{column}_indx"], outputCols=[f"{column}_ohe"])
        encoders.append(encoder)
pipeline = Pipeline(stages=indexers + encoders)

# Stosowanie transformacji
new_data = pipeline.fit(data).transform(data)

        

In [ ]:
import statsmodels.formula.api as smf
from pyspark.sql.functions import col

In [ ]:
new_data_pandas = new_data.select( 'Age', 
 'Gender_ohe',
 'Marital Status_ohe',
 'Occupation_ohe',
 'Monthly Income_ohe',
 'Educational Qualifications_ohe',
 'Family size',
 'latitude',
 'longitude',
 'Pin code',
 'Output_ohe',
 'Feedback_ohe',
 '_c12')

In [ ]:
pandas_df = new_data_pandas.toPandas()

In [ ]:
from pyspark.sql.functions import when

In [ ]:
new_data = new_data.withColumn("_c12", when(new_data["_c12"] == "Yes", 1).otherwise(0))

In [ ]:
def forward_selection(data, response):
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {}".format(response, ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().aic
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop(0)
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {}".format(response, ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model, selected
model, selected_columns = forward_selection(pandas_df, '_c12')

In [ ]:
assembler = VectorAssembler(inputCols=['Output_ohe', 'Gender_ohe', 'Monthly Income_ohe'], outputCol='features')

In [ ]:
new_data.head()

In [ ]:
trans = assembler.transform(new_data) 

In [ ]:
trans = trans.withColumn("_c12", col("_c12").cast("int"))

In [ ]:
trans.head()

In [ ]:
mm = trans.select('features', '_c12')

In [ ]:
mm = mm.withColumn("_c12", col("_c12").cast("int"))

In [ ]:
train_data, test_data = mm.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(featuresCol='features', labelCol='_c12')

In [ ]:
modelus = lr.fit(train_data)

In [ ]:
results = modelus.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
predictions = BinaryClassificationEvaluator(labelCol='_c12', rawPredictionCol='prediction')

In [ ]:
auc = predictions.evaluate(results)

In [7]:
auc

NameError: name 'auc' is not defined